In [40]:
import numpy as np
import pandas as pd
import re
#import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
# Decision tree
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

from sklearn.model_selection import GridSearchCV

In [41]:
# 파일 읽기
train = pd.read_csv('crop_train.csv')
test = pd.read_csv('crop_test.csv')

# 제출 형식에 따라 ID 추가
train['ID'] = range(0, len(train))
test['ID'] = range(0, len(test))

# 결과값 넣을 set 준비
result = test['ID']

train = train.drop(['ID'], axis = 1)
test = test.drop(['ID'], axis = 1)

#복사본 생성
original_train = train.copy()
original_test = test.copy()
full_data = [train, test]

train

,A,P,L1,L2,Ec,C,EqD,Ex,S,SF,Class
0,78522.0,1076.493,430.966628,233.047703,0.841180,79158.0,316.191897,0.816101,0.991965,0.538287,1
1,210260.0,1739.133,648.610364,417.898925,0.764774,213691.0,517.408298,0.823341,0.983944,0.636354,5
2,46566.0,819.442,319.762455,186.820417,0.811576,46990.0,243.494708,0.648913,0.990977,0.579861,4
3,74795.0,1034.077,393.361325,243.115326,0.786142,75468.0,308.596746,0.802435,0.991082,0.615459,1
4,54801.0,871.432,327.957559,213.095150,0.760136,55357.0,264.149201,0.730203,0.989956,0.648729,2
...,...,...,...,...,...,...,...,...,...,...,...
9995,27050.0,607.241,220.980437,156.430219,0.706321,27365.0,185.583215,0.719032,0.988489,0.705293,6
9996,32836.0,669.190,218.406004,191.758773,0.478675,33225.0,204.470276,0.802915,0.988292,0.876458,2
9997,70908.0,1018.387,401.155356,226.730093,0.824959,71865.0,300.471080,0.800388,0.986683,0.561022,1
9998,38470.0,762.336,304.588947,162.011649,0.846806,38914.0,221.317702,0.801826,0.988590,0.527964,3


In [42]:
original_test['ID'] = range(0, len(test))
original_test['Class'] = -1

def assign_class(row):
    if row['EqD'] > 390 and row['C'] > 120000:
        return 5
    if row['Ec'] < 0.6 and row['SF'] > 0.84:
        return 2
    if row['SF'] < 0.46:
        return 3
    if row['SF'] > 0.63 and row['L2'] <170:
        return 6
    else:
        return -1
original_test['Class'] = original_test.apply(assign_class, axis=1)
columns_to_drop = [col for col in test.columns if col not in ['ID', 'Class']]
original_test.drop(columns=columns_to_drop, inplace=True)

In [43]:
'''
plt.figure(figsize=(10,5))
sns.kdeplot(train[train['Class'] == 6]['A'])
sns.kdeplot(train[train['Class'] == 5]['A'])
sns.kdeplot(train[train['Class'] == 4]['A'])
sns.kdeplot(train[train['Class'] == 3]['A'])
sns.kdeplot(train[train['Class'] == 2]['A'])
sns.kdeplot(train[train['Class'] == 1]['A'])
sns.kdeplot(train[train['Class'] == 0]['A'])
plt.legend(['Class == 6', 'Class == 5', 'Class == 4', 'Class == 3', 'Class == 2', 'Class == 1', 'Class == 0'])
'''

"\nplt.figure(figsize=(10,5))\nsns.kdeplot(train[train['Class'] == 6]['A'])\nsns.kdeplot(train[train['Class'] == 5]['A'])\nsns.kdeplot(train[train['Class'] == 4]['A'])\nsns.kdeplot(train[train['Class'] == 3]['A'])\nsns.kdeplot(train[train['Class'] == 2]['A'])\nsns.kdeplot(train[train['Class'] == 1]['A'])\nsns.kdeplot(train[train['Class'] == 0]['A'])\nplt.legend(['Class == 6', 'Class == 5', 'Class == 4', 'Class == 3', 'Class == 2', 'Class == 1', 'Class == 0'])\n"

In [44]:
'''
plt.figure(figsize=(12,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, linecolor='white', annot=True)
'''

"\nplt.figure(figsize=(12,12))\nplt.title('Pearson Correlation of Features', y=1.05, size=15)\nsns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, linecolor='white', annot=True)\n"

In [45]:

def is_outliers(s):
    lower_limit = s.mean() - (s.std()*5)
    upper_limit = s.mean() + (s.std()*5)
    return ~s.between(lower_limit, upper_limit)

train = train[~train.groupby('Class')['A'].apply(is_outliers)]
train = train[~train.groupby('Class')['P'].apply(is_outliers)]
train = train[~train.groupby('Class')['L1'].apply(is_outliers)]
train = train[~train.groupby('Class')['L2'].apply(is_outliers)]
train = train[~train.groupby('Class')['Ec'].apply(is_outliers)]
train = train[~train.groupby('Class')['C'].apply(is_outliers)]
train = train[~train.groupby('Class')['EqD'].apply(is_outliers)]
train = train[~train.groupby('Class')['Ex'].apply(is_outliers)]
train = train[~train.groupby('Class')['S'].apply(is_outliers)]
train = train[~train.groupby('Class')['SF'].apply(is_outliers)]
print(train.shape)



(9959, 11)


In [46]:
# nomalizer, standardscaler, minmaxscaler 중에 nomarlizer가 성능 가장 좋다
y_train = train['Class']
train = train.drop(['Class'], axis=1)

#features = [['A', 'P', 'L1', 'L2', 'Ec', 'C', 'EqD', 'Ex', 'S', 'SF']]
features = [['A', 'P', 'L1', 'L2', 'Ec', 'C', 'EqD', 'Ex', 'S', 'SF']]

#x_train_s = train
#x_train_m = train
'''
scaler = Normalizer().fit(train)
x_train_s = scaler.transform(train)
scaler = Normalizer().fit(test)
x_test_s = scaler.transform(test)
'''
scaler = StandardScaler()
x_train_s = scaler.fit_transform(train)
scaler = StandardScaler()
x_test_s = scaler.fit_transform(test)
'''
scaler = MinMaxScaler().fit(train)
x_train_s = scaler.transform(train)
scaler = MinMaxScaler().fit(test)
x_test_s = scaler.transform(test)
'''

'''
x_train_s = train.copy()
x_test_s = test.copy()

scaler = StandardScaler()

for feature in features:
	x_train_s[feature] = scaler.fit_transform(train[feature])
	x_test_s[feature] = scaler.fit_transform(test[feature])
	'''


'\nx_train_s = train.copy()\nx_test_s = test.copy()\n\nscaler = StandardScaler()\n\nfor feature in features:\n\tx_train_s[feature] = scaler.fit_transform(train[feature])\n\tx_test_s[feature] = scaler.fit_transform(test[feature])\n\t'

In [47]:

'''
#model= KNeighborsClassifier(metric= 'euclidean', n_neighbors= 8, weights= 'distance')
model = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 17, weights= 'uniform')
model.fit(x_train_s,y_train)
prediction = model.predict(x_test_s)

print(model.score(x_train_s,y_train))

sub = pd.DataFrame({"ID": result,"Class": prediction})

i = 0  # 초기값

for index, row in original_test.iterrows():
    if row['Class'] != -1 and sub.loc[sub['ID'] == row['ID'], 'Class'].values[0] != row['Class']:
        i += 1  # 두 표의 'Class' 값이 다를 때 i 증가
        sub.loc[sub['ID'] == row['ID'], 'Class'] = row['Class']

print("Number of rows with different 'Class' values:", i)


sub.to_csv('submission.csv', index=False)
'''

'\n#model= KNeighborsClassifier(metric= \'euclidean\', n_neighbors= 8, weights= \'distance\')\nmodel = KNeighborsClassifier(metric= \'manhattan\', n_neighbors= 17, weights= \'uniform\')\nmodel.fit(x_train_s,y_train)\nprediction = model.predict(x_test_s)\n\nprint(model.score(x_train_s,y_train))\n\nsub = pd.DataFrame({"ID": result,"Class": prediction})\n\ni = 0  # 초기값\n\nfor index, row in original_test.iterrows():\n    if row[\'Class\'] != -1 and sub.loc[sub[\'ID\'] == row[\'ID\'], \'Class\'].values[0] != row[\'Class\']:\n        i += 1  # 두 표의 \'Class\' 값이 다를 때 i 증가\n        sub.loc[sub[\'ID\'] == row[\'ID\'], \'Class\'] = row[\'Class\']\n\nprint("Number of rows with different \'Class\' values:", i)\n\n\nsub.to_csv(\'submission.csv\', index=False)\n'

In [48]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
'''
model = TSNE()
transform1 = model.fit_transform(x_train_s, y_train)
transform2 = model.fit_transform(x_test_s)


pca = PCA()
transform1 = pca.fit_transform(x_train_s, y_train)
transform2 = pca.fit_transform(x_test_s)
'''

lda = LinearDiscriminantAnalysis()
transform1 = lda.fit_transform(x_train_s, y_train)
transform2 = lda.transform(x_test_s)

In [49]:


#model= KNeighborsClassifier(metric= 'euclidean', n_neighbors= 8, weights= 'distance')
#model = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 17, weights= 'uniform') 'metric': 'manhattan', 'n_neighbors': 13, 'weights': 'distance'
model = KNeighborsClassifier(metric= 'euclidean', n_neighbors= 14, weights= 'distance')
model.fit(transform1, y_train)
prediction = model.predict(transform2)

print(model.score(transform1,y_train))

sub = pd.DataFrame({"ID": result,"Class": prediction})

i = 0  # 초기값

for index, row in original_test.iterrows():
    if row['Class'] != -1 and sub.loc[sub['ID'] == row['ID'], 'Class'].values[0] != row['Class']:
        i += 1  # 두 표의 'Class' 값이 다를 때 i 증가
        sub.loc[sub['ID'] == row['ID'], 'Class'] = row['Class']

print("Number of rows with different 'Class' values:", i)


sub.to_csv('submission.csv', index=False)

1.0
Number of rows with different 'Class' values: 1


In [50]:
'''
#y = train['Class']
#X = train.drop(['Class'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, random_state=30)

train_acc = []
test_acc = []

for n in range(1,15):
    clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=n)
    clf.fit(x_train, y_train)
    prediction = clf.predict(x_test)
    train_acc.append(clf.score(x_train, y_train))
    test_acc.append((prediction==y_test).mean())
    
plt.figure(figsize=(12, 9))
plt.plot(range(1, 15), train_acc, label='TRAIN set')
plt.plot(range(1, 15), test_acc, label='TEST set')
plt.xlabel("n_neighbors")
plt.ylabel("accuracy")
plt.xticks(np.arange(0, 16, step=1))
plt.legend()
'''

'\n#y = train[\'Class\']\n#X = train.drop([\'Class\'], axis=1)\n\nx_train, x_test, y_train, y_test = train_test_split(x_train, y_train, random_state=30)\n\ntrain_acc = []\ntest_acc = []\n\nfor n in range(1,15):\n    clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=n)\n    clf.fit(x_train, y_train)\n    prediction = clf.predict(x_test)\n    train_acc.append(clf.score(x_train, y_train))\n    test_acc.append((prediction==y_test).mean())\n    \nplt.figure(figsize=(12, 9))\nplt.plot(range(1, 15), train_acc, label=\'TRAIN set\')\nplt.plot(range(1, 15), test_acc, label=\'TEST set\')\nplt.xlabel("n_neighbors")\nplt.ylabel("accuracy")\nplt.xticks(np.arange(0, 16, step=1))\nplt.legend()\n'

In [51]:
x_train, x_test, y_train, y_test = train_test_split(transform1, y_train, random_state=30)
K = 50
acc = np.zeros([K])

for n in range(1, K+1):
    classifier = KNeighborsClassifier(n_neighbors=n)
    classifier.fit(x_train, y_train.ravel())
    y_pred = classifier.predict(x_test)
    acc[n-1] = metrics.accuracy_score(y_test, y_pred)
print(acc)
print(f'Maximum: {np.max(acc)}')
print(f'Minimum: {np.min(acc)}')
max_accuracy_index = np.argmax(acc) + 1
print(f'N value for Maximum Accuracy: {max_accuracy_index}')

[0.91485944 0.89799197 0.92449799 0.92409639 0.92610442 0.9248996
 0.9248996  0.9248996  0.9248996  0.92730924 0.92610442 0.9253012
 0.92329317 0.9253012  0.9248996  0.92610442 0.92610442 0.9253012
 0.9248996  0.92570281 0.92248996 0.92329317 0.92208835 0.92208835
 0.92369478 0.92409639 0.92409639 0.92570281 0.92409639 0.9248996
 0.92409639 0.9248996  0.92449799 0.92449799 0.92369478 0.92369478
 0.92449799 0.92329317 0.92449799 0.9253012  0.92449799 0.9248996
 0.9253012  0.92570281 0.92650602 0.9253012  0.92650602 0.92570281
 0.92650602 0.92650602]
Maximum: 0.9273092369477912
Minimum: 0.8979919678714859
N value for Maximum Accuracy: 10


In [52]:

grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski'],
}

gs = GridSearchCV(model, grid_params, cv=10)
gs.fit(x_train, y_train)
print("Best Parameters : ", gs.best_params_)
print("Best Score : ", gs.best_score_)
print("Best Test Score : ", gs.score(x_test, y_test))

'''
knn_s = KNeighborsClassifier()
knn_s.fit(x_train_s, y_train)

gs = GridSearchCV(model, grid_params, cv=10)
gs.fit(x_train_s, y_train)
print("Best Parameters : ", gs.best_params_)
print("Best Score : ", gs.best_score_)
print("Best Test Score : ", gs.score(x_test, y_test))

gs = GridSearchCV(model, grid_params, cv=10)
gs.fit(x_train_m, y_train)
print("Best Parameters : ", gs.best_params_)
print("Best Score : ", gs.best_score_)
print("Best Test Score : ", gs.score(x_test, y_test))
'''

Best Parameters :  {'metric': 'euclidean', 'n_neighbors': 14, 'weights': 'distance'}
Best Score :  0.9231517670323834
Best Test Score :  0.9244979919678715


'\nknn_s = KNeighborsClassifier()\nknn_s.fit(x_train_s, y_train)\n\ngs = GridSearchCV(model, grid_params, cv=10)\ngs.fit(x_train_s, y_train)\nprint("Best Parameters : ", gs.best_params_)\nprint("Best Score : ", gs.best_score_)\nprint("Best Test Score : ", gs.score(x_test, y_test))\n\ngs = GridSearchCV(model, grid_params, cv=10)\ngs.fit(x_train_m, y_train)\nprint("Best Parameters : ", gs.best_params_)\nprint("Best Score : ", gs.best_score_)\nprint("Best Test Score : ", gs.score(x_test, y_test))\n'